# AutoInt+ 모델 [프로젝트]

In [158]:
# 라이브러리 호출
import time
import random
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, MaxPooling2D, Conv2D, Dropout, Lambda, Dense, Flatten, Activation, Input, Embedding, BatchNormalization
from tensorflow.keras.initializers import glorot_normal, Zeros, TruncatedNormal
from tensorflow.keras.regularizers import l2


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy


from tensorflow.keras.optimizers import Adam
from collections import defaultdict
import math
import joblib 

import warnings
warnings.filterwarnings('ignore')

## Layer 정의

### 임베딩 레이어

In [159]:
class FeaturesEmbedding(Layer):  
    '''
    임베딩 레이어입니다. 
    - 만약 피처(feature) 3개가 각각 10개, 20개, 30개의 고유값을 가진다면 feature_dims는 [10, 20, 30] 형태를 띄게 됩니다.
    - 전체 임베딩을 해야 할 개수는 10+20+30 = 60이므로 '60 x 임베딩_차원_크기'의 행렬이 생성되게 됩니다.
    '''
    def __init__(self, field_dims, embed_dim, **kwargs):
        super(FeaturesEmbedding, self).__init__(**kwargs)
        self.total_dim = sum(field_dims)
        self.embed_dim = embed_dim
        ## int 64로 바꿔주자
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)
        self.embedding = tf.keras.layers.Embedding(input_dim=self.total_dim, output_dim=self.embed_dim)

    def build(self, input_shape):
        # 임베딩을 빌드하고 초기화합니다.
        self.embedding.build(input_shape)
        self.embedding.set_weights([tf.keras.initializers.GlorotUniform()(shape=self.embedding.weights[0].shape)])

    def call(self, x):
        # 들어온 입력의 임베딩을 가져니다.
        x = x + tf.constant(self.offsets)
        return self.embedding(x)

### 다층 퍼셉트론

In [160]:

class MultiLayerPerceptron(Layer):  
    '''
    DNN 레이어입니다.
    - Tensorflow Keras에서는 Dense 레이어를 쌓아올린 구조입니다.
    - 필요에 따라 배치 정규화도 사용할 수 있습니다.
    '''
    def __init__(self, input_dim, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_bn=False, init_std=0.0001, output_layer=True):
        super(MultiLayerPerceptron, self).__init__()
        self.dropout_rate = dropout_rate
        self.use_bn = use_bn
        hidden_units = [input_dim] + list(hidden_units)
        if output_layer:
            hidden_units += [1]
        # Dense layer를 쌓아올립니다.
        self.linears = [Dense(units, activation=None, kernel_initializer=tf.random_normal_initializer(stddev=init_std),
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg)) for units in hidden_units[1:]]
        # 활성화 함수를 세팅합니다.
        self.activation = tf.keras.layers.Activation(activation)
        # 필요하다면 배치정규화도 진행합니다.
        if self.use_bn:
            self.bn = [BatchNormalization() for _ in hidden_units[1:]]
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        x = inputs
        for i in range(len(self.linears)):
            # input data가 들어오면 layer를 돌면서 벡터 값을 가져오게 됩니다.
            x = self.linears[i](x)
            if self.use_bn:
                x = self.bn[i](x, training=training)
            # 각 layer마다 나온 벡터 값에 활성화 함수와 dropout을 적용시켜 비선형성 구조와 과적합을 방지합니다.
            x = self.activation(x)
            x = self.dropout(x, training=training)
        return x

### 멀티 헤드 어텐션

In [161]:
class MultiHeadSelfAttention(Layer):  
    '''
    멀티 헤드 셀프 어텐션 레이어입니다.
    - 위에 작성한 수식과 같이 동작됩니다.
    - 필요에 따라 잔차 연결(residual connection)도 진행합니다.
    '''
    def __init__(self, att_embedding_size=8, head_num=2, use_res=True, scaling=False, seed=1024, **kwargs):
        if head_num <= 0:
            raise ValueError('head_num must be a int > 0')
        self.att_embedding_size = att_embedding_size
        self.head_num = head_num
        self.use_res = use_res
        self.seed = seed
        self.scaling = scaling
        super(MultiHeadSelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions" % (len(input_shape)))
        embedding_size = int(input_shape[-1])
        # 쿼리에 해당하는 매트릭스입니다. 
        self.W_Query = self.add_weight(name='query', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed))
        # 키에 해당되는 매트릭스입니다.
        self.W_key = self.add_weight(name='key', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                     dtype=tf.float32,
                                     initializer=TruncatedNormal(seed=self.seed + 1))
        # 값(value)에 해당되는 매트릭스입니다.
        self.W_Value = self.add_weight(name='value', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed + 2))
        # 필요하다면 잔차 연결도 할 수 있습니다.
        if self.use_res:
            self.W_Res = self.add_weight(name='res', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                         dtype=tf.float32,
                                         initializer=TruncatedNormal(seed=self.seed))

        super(MultiHeadSelfAttention, self).build(input_shape)

    def call(self, inputs, **kwargs):
        if K.ndim(inputs) != 3:
            raise ValueError("Unexpected inputs dimensions %d, expect to be 3 dimensions" % (K.ndim(inputs)))
        
        # 입력이 들어오면 쿼리, 키, 값(value)에 매칭되어 각각의 값을 가지고 옵니다.
        querys = tf.tensordot(inputs, self.W_Query, axes=(-1, 0))  
        keys = tf.tensordot(inputs, self.W_key, axes=(-1, 0))
        values = tf.tensordot(inputs, self.W_Value, axes=(-1, 0))

        # 헤드 개수에 따라 데이터를 분리해줍니다.
        querys = tf.stack(tf.split(querys, self.head_num, axis=2))
        keys = tf.stack(tf.split(keys, self.head_num, axis=2))
        values = tf.stack(tf.split(values, self.head_num, axis=2))
        
        # 쿼리와 키를 먼저 곱해줍니다. 위 이미지의 식 (5)와 같습니다.
        inner_product = tf.matmul(querys, keys, transpose_b=True)
        if self.scaling:
            inner_product /= self.att_embedding_size ** 0.5
        self.normalized_att_scores =  tf.nn.softmax(inner_product)
        
        # 쿼리와 키에서 나온 어텐션 값을 값(value)에 곱해줍니다. 식 (6)과 같습니다.
        result = tf.matmul(self.normalized_att_scores, values)
        # 식 (7)과 같이 쪼개어진 멀테 헤드를 모아줍니다.
        result = tf.concat(tf.split(result, self.head_num, ), axis=-1)
        result = tf.squeeze(result, axis=0) 

        if self.use_res:
            result += tf.tensordot(inputs, self.W_Res, axes=(-1, 0))
        result = tf.nn.relu(result)
        
        # 그 결과 값을 리턴합니다.

        return result

    def compute_output_shape(self, input_shape):

        return (None, input_shape[1], self.att_embedding_size * self.head_num)

    def get_config(self, ):
        config = {'att_embedding_size': self.att_embedding_size, 'head_num': self.head_num, 'use_res': self.use_res,'seed': self.seed}
        base_config = super(MultiHeadSelfAttention, self).get_config()
        base_config.update(config)
        return base_config

## AutoInt+ 모델

In [162]:
class AutoIntMLP(Layer): 
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2, att_res=True, dnn_hidden_units=(32, 32), dnn_activation='relu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False, dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntMLP, self).__init__()
        self.embedding = FeaturesEmbedding(field_dims, embedding_size)
        self.num_fields = len(field_dims)
        self.embedding_size = embedding_size

        self.final_layer = Dense(1, use_bias=False, kernel_initializer=tf.random_normal_initializer(stddev=init_std))
        
        self.dnn = tf.keras.Sequential()
        for units in dnn_hidden_units:
            self.dnn.add(Dense(units, activation=dnn_activation,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_reg_dnn),
                               kernel_initializer=tf.random_normal_initializer(stddev=init_std)))
            if dnn_use_bn:
                self.dnn.add(BatchNormalization())
            self.dnn.add(Activation(dnn_activation))
            if dnn_dropout > 0:
                self.dnn.add(Dropout(dnn_dropout))
        self.dnn.add(Dense(1, kernel_initializer=tf.random_normal_initializer(stddev=init_std)))

        self.int_layers = [MultiHeadSelfAttention(att_embedding_size=embedding_size, head_num=att_head_num, use_res=att_res) for _ in range(att_layer_num)]

    def call(self, inputs):
        embed_x = self.embedding(inputs)
        dnn_embed = tf.reshape(embed_x, shape=(-1, self.embedding_size * self.num_fields))

        att_input = embed_x
        for layer in self.int_layers:
            att_input = layer(att_input)

        att_output = Flatten()(att_input)
        att_output = self.final_layer(att_output)
        
        dnn_output = self.dnn(dnn_embed)
        y_pred = tf.keras.activations.sigmoid(att_output + dnn_output)
        
        return y_pred

## 평가 지표

In [163]:
def get_DCG(ranklist, y_true):
    dcg = 0.0
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item in y_true:
            dcg += 1.0 / math.log(i + 2)
    return  dcg

def get_IDCG(ranklist, y_true):
    idcg = 0.0
    i = 0
    for item in y_true:
        if item in ranklist:
            idcg += 1.0 / math.log(i + 2)
            i += 1
    return idcg

def get_NDCG(ranklist, y_true):
    '''NDCG 평가 지표'''
    ranklist = np.array(ranklist).astype(int)
    y_true = np.array(y_true).astype(int)
    dcg = get_DCG(ranklist, y_true)
    idcg = get_IDCG(y_true, y_true)
    if idcg == 0:
        return 0
    return round( (dcg / idcg), 5)

def get_hit_rate(ranklist, y_true):
    '''hitrate 평가 지표'''
    c = 0
    for y in y_true:
        if y in ranklist:
            c += 1
    return round( c / len(y_true), 5 )

## 모델 테스트

In [164]:

def test_model(model, test_df):
    '''모델 테스트'''
    user_pred_info = defaultdict(list)
    total_rows = len(test_df)
    for i in range(0, total_rows, batch_size):
        features = test_df.iloc[i:i + batch_size, :-1].astype(np.int32).values  # 데이터 타입 변경 
        y_pred = model.predict(features, verbose=False)
        for feature, p in zip(features, y_pred):
            u_i = feature[:2]
            user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
    return user_pred_info

## 학습 데이터 생성

In [165]:
project_path = os.path.abspath(os.getcwd())
data_dir_nm = 'data'
movielens_dir_nm = 'ml-1m'
model_dir_nm = 'model'
data_path = f"{project_path}/{data_dir_nm}"

In [166]:
# 1. 데이터 불러오기
# csv 데이터이므로 read_csv로 가져옵니다.
movielens_rcmm = pd.read_csv(f"{data_path}/movielens_rcmm_v2.csv", dtype=str)
print(movielens_rcmm.shape)
movielens_rcmm.head()

(1000209, 15)


,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,rating_decade,genre1,genre2,genre3,gender,age,occupation,zip,label
0,1,1193,1970s,1975,2000,12,2000s,Drama,no,no,F,1,10,48067,1
1,1,661,1990s,1996,2000,12,2000s,Animation,Children's,Musical,F,1,10,48067,0
2,1,914,1960s,1964,2000,12,2000s,Musical,Romance,no,F,1,10,48067,0
3,1,3408,2000s,2000,2000,12,2000s,Drama,no,no,F,1,10,48067,1
4,1,2355,1990s,1998,2001,1,2000s,Animation,Children's,Comedy,F,1,10,48067,1


In [167]:
# 2. 라벨 인코더(label encoder)
# label은 제외한 각 컬럼을 돌면서 각각의 고윳값들을 0부터 n까지 매핑시킵니다.
label_encoders = {col: LabelEncoder() for col in movielens_rcmm.columns[:-1]} # label은 제외

for col, le in label_encoders.items():
    movielens_rcmm[col] = le.fit_transform(movielens_rcmm[col])

In [168]:
movielens_rcmm.head()

,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,rating_decade,genre1,genre2,genre3,gender,age,occupation,zip,label
0,0,189,6,55,0,3,0,7,17,15,0,0,2,1588,1
1,0,3374,8,76,0,3,0,2,2,8,0,0,2,1588,0
2,0,3615,5,44,0,3,0,11,12,15,0,0,2,1588,0
3,0,2503,9,80,0,3,0,7,17,15,0,0,2,1588,1
4,0,1374,8,78,1,0,0,2,2,2,0,0,2,1588,1


In [169]:
movielens_rcmm['label'] = movielens_rcmm['label'].astype(np.float32)

In [170]:

# 3. 학습 데이터와 테스트데이터로 분리, 0.2 정도로 분리
train_df, test_df = train_test_split(movielens_rcmm, test_size=0.2, random_state=42)

In [171]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800167 entries, 416292 to 121958
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user_id        800167 non-null  int64  
 1   movie_id       800167 non-null  int64  
 2   movie_decade   800167 non-null  int64  
 3   movie_year     800167 non-null  int64  
 4   rating_year    800167 non-null  int64  
 5   rating_month   800167 non-null  int64  
 6   rating_decade  800167 non-null  int64  
 7   genre1         800167 non-null  int64  
 8   genre2         800167 non-null  int64  
 9   genre3         800167 non-null  int64  
 10  gender         800167 non-null  int64  
 11  age            800167 non-null  int64  
 12  occupation     800167 non-null  int64  
 13  zip            800167 non-null  int64  
 14  label          800167 non-null  float32
dtypes: float32(1), int64(14)
memory usage: 94.6 MB


In [172]:
# 필요 컬럼들과 레이블 정의
# 필드의 각 고유 개수를 정의하는 field_dims를 정의합니다. 이는  임베딩 때 활용됩니다. 
u_i_feature = ['user_id', 'movie_id']
meta_features = ['movie_decade', 'movie_year', 'rating_year', 'rating_month', 'rating_decade', 'genre1','genre2', 'genre3', 'gender', 'age', 'occupation', 'zip']
label = 'label'
field_dims = np.max(movielens_rcmm[u_i_feature + meta_features].astype(np.int64).values, axis=0) + 1
field_dims

array([6040, 3706,   10,   81,    4,   12,    1,   18,   18,   16,    2,
          7,   21, 3439])

## 훈련 환경 및 모델 세팅 -> 학습

In [173]:
# # 에포크, 학습률, 드롭아웃, 배치사이즈, 임베딩 크기 등 정의
# epochs=5
# learning_rate= 0.0001
# dropout= 0.4
# batch_size = 2048
# embed_dim= 16

# 실험하고 싶은 하이퍼파라미트 리스트 정의 - 어텐션 헤드 수 파라미터 추가
experiment_configs = [
    {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 2}, # 기본
    {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 4}, # 어텐션 헤드 수 4
    {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 8}, # 어텐션 헤드 수 8
    {'epochs': 3, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 32, 'att_head_num': 4},
    {'epochs': 5, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 16, 'att_head_num': 8},
    {'epochs': 5, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 32, 'att_head_num': 8}, # 임베딩 사이즈 증가
    {'epochs': 5, 'learning_rate': 0.0005, 'dropout': 0.3, 'batch_size': 2048, 'embed_dim': 8, 'att_head_num': 4},
    {'epochs': 5, 'learning_rate': 0.0005, 'dropout': 0.3, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 8},
    {'epochs': 3, 'learning_rate': 0.0001, 'dropout': 0.7, 'batch_size': 1024, 'embed_dim': 16, 'att_head_num': 2},
    {'epochs': 3, 'learning_rate': 0.0001, 'dropout': 0.7, 'batch_size': 1024, 'embed_dim': 16, 'att_head_num': 4}    
]

In [174]:
# AutoInt+ 레이어를 가지고 있는 모델 본체
class AutoIntMLPModel(Model):
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2,
                 att_res=True, dnn_hidden_units=(32, 32), dnn_activation='relu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False,
                 dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntMLPModel, self).__init__()
        self.autoInt_layer = AutoIntMLP(
            field_dims=field_dims,
            embedding_size=embedding_size,
            att_layer_num=att_layer_num,
            att_head_num=att_head_num,
            att_res=att_res,
            dnn_hidden_units=dnn_hidden_units,
            dnn_activation=dnn_activation,
            l2_reg_dnn=l2_reg_dnn,
            l2_reg_embedding=l2_reg_embedding,
            dnn_use_bn=dnn_use_bn,
            dnn_dropout=dnn_dropout,
            init_std=init_std
        )

    def call(self, inputs, training=False):
        return self.autoInt_layer(inputs, training=training)

In [175]:
# 모델 정의 -> 학습 반복
trained_models = []
model_configs = []

for i, config in enumerate(experiment_configs):
    print(f"\n▶ Experiment {i+1}: {config}")

    autoIntMLP_model = AutoIntMLPModel(
        field_dims=field_dims,
        embedding_size=config['embed_dim'],
        att_layer_num=3,
        att_head_num=config['att_head_num'],
        att_res=True,
        dnn_hidden_units=(32, 32),               # 추가: DNN 은닉층 구조
        dnn_activation='relu',                  # 추가: 활성화 함수
        l2_reg_dnn=0,
        l2_reg_embedding=1e-5,
        dnn_use_bn=False,
        dnn_dropout=config['dropout'],
        init_std=0.0001
        )

    autoIntMLP_model.compile(
        optimizer=Adam(learning_rate=config['learning_rate']),
        loss=BinaryCrossentropy(from_logits=False),
        metrics=['binary_crossentropy']
    )

    X = train_df[u_i_feature + meta_features].astype(np.int32)
    y = train_df[label]

    autoIntMLP_model.fit(X, y,
              epochs=config['epochs'],
              batch_size=config['batch_size'],
              validation_split=0.1
              )

    # 저장용 리스트에 모델과 설정 기록
    trained_models.append(autoIntMLP_model)
    model_configs.append(config)

print("\n 모든 실험 학습 완료.")


▶ Experiment 1: {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 2}
Epoch 1/5
352/352 [==============================] - 9s 23ms/step - loss: 0.6731 - binary_crossentropy: 0.6731 - val_loss: 0.6292 - val_binary_crossentropy: 0.6292
Epoch 2/5
352/352 [==============================] - 8s 22ms/step - loss: 0.6025 - binary_crossentropy: 0.6025 - val_loss: 0.5833 - val_binary_crossentropy: 0.5833
Epoch 3/5
352/352 [==============================] - 8s 24ms/step - loss: 0.5611 - binary_crossentropy: 0.5611 - val_loss: 0.5484 - val_binary_crossentropy: 0.5484
Epoch 4/5
352/352 [==============================] - 8s 23ms/step - loss: 0.5397 - binary_crossentropy: 0.5397 - val_loss: 0.5418 - val_binary_crossentropy: 0.5418
Epoch 5/5
352/352 [==============================] - 8s 24ms/step - loss: 0.5348 - binary_crossentropy: 0.5348 - val_loss: 0.5396 - val_binary_crossentropy: 0.5396



▶ Experiment 2: {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 4}
Epoch 1/5
352/352 [==============================] - 17s 46ms/step - loss: 0.6589 - binary_crossentropy: 0.6589 - val_loss: 0.6047 - val_binary_crossentropy: 0.6047
Epoch 2/5
352/352 [==============================] - 16s 46ms/step - loss: 0.5817 - binary_crossentropy: 0.5817 - val_loss: 0.5552 - val_binary_crossentropy: 0.5552
Epoch 3/5
352/352 [==============================] - 16s 46ms/step - loss: 0.5424 - binary_crossentropy: 0.5424 - val_loss: 0.5419 - val_binary_crossentropy: 0.5419
Epoch 4/5
352/352 [==============================] - 17s 48ms/step - loss: 0.5351 - binary_crossentropy: 0.5351 - val_loss: 0.5397 - val_binary_crossentropy: 0.5397
Epoch 5/5
352/352 [==============================] - 17s 48ms/step - loss: 0.5327 - binary_crossentropy: 0.5327 - val_loss: 0.5387 - val_binary_crossentropy: 0.5387



▶ Experiment 3: {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 8}
Epoch 1/5
352/352 [==============================] - 38s 106ms/step - loss: 0.6464 - binary_crossentropy: 0.6464 - val_loss: 0.5830 - val_binary_crossentropy: 0.5830
Epoch 2/5
352/352 [==============================] - 36s 102ms/step - loss: 0.5502 - binary_crossentropy: 0.5502 - val_loss: 0.5422 - val_binary_crossentropy: 0.5422
Epoch 3/5
352/352 [==============================] - 35s 100ms/step - loss: 0.5359 - binary_crossentropy: 0.5359 - val_loss: 0.5396 - val_binary_crossentropy: 0.5396
Epoch 4/5
352/352 [==============================] - 35s 100ms/step - loss: 0.5331 - binary_crossentropy: 0.5331 - val_loss: 0.5380 - val_binary_crossentropy: 0.5380
Epoch 5/5
352/352 [==============================] - 35s 100ms/step - loss: 0.5319 - binary_crossentropy: 0.5319 - val_loss: 0.5389 - val_binary_crossentropy: 0.5389



▶ Experiment 4: {'epochs': 3, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 32, 'att_head_num': 4}
Epoch 1/3
704/704 [==============================] - 37s 52ms/step - loss: 0.5617 - binary_crossentropy: 0.5617 - val_loss: 0.5362 - val_binary_crossentropy: 0.5362
Epoch 2/3
704/704 [==============================] - 36s 51ms/step - loss: 0.5306 - binary_crossentropy: 0.5306 - val_loss: 0.5314 - val_binary_crossentropy: 0.5314
Epoch 3/3
704/704 [==============================] - 36s 52ms/step - loss: 0.5212 - binary_crossentropy: 0.5212 - val_loss: 0.5249 - val_binary_crossentropy: 0.5249



▶ Experiment 5: {'epochs': 5, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 16, 'att_head_num': 8}
Epoch 1/5
704/704 [==============================] - 40s 55ms/step - loss: 0.5632 - binary_crossentropy: 0.5632 - val_loss: 0.5381 - val_binary_crossentropy: 0.5381
Epoch 2/5
704/704 [==============================] - 40s 57ms/step - loss: 0.5329 - binary_crossentropy: 0.5329 - val_loss: 0.5316 - val_binary_crossentropy: 0.5316
Epoch 3/5
704/704 [==============================] - 39s 56ms/step - loss: 0.5243 - binary_crossentropy: 0.5243 - val_loss: 0.5266 - val_binary_crossentropy: 0.5266
Epoch 4/5
704/704 [==============================] - 40s 57ms/step - loss: 0.5168 - binary_crossentropy: 0.5168 - val_loss: 0.5225 - val_binary_crossentropy: 0.5225
Epoch 5/5
704/704 [==============================] - 40s 57ms/step - loss: 0.5088 - binary_crossentropy: 0.5088 - val_loss: 0.5201 - val_binary_crossentropy: 0.5201



▶ Experiment 6: {'epochs': 5, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 32, 'att_head_num': 8}
Epoch 1/5
704/704 [==============================] - 91s 128ms/step - loss: 0.5599 - binary_crossentropy: 0.5599 - val_loss: 0.5353 - val_binary_crossentropy: 0.5353
Epoch 2/5
704/704 [==============================] - 91s 129ms/step - loss: 0.5299 - binary_crossentropy: 0.5299 - val_loss: 0.5285 - val_binary_crossentropy: 0.5285
Epoch 3/5
704/704 [==============================] - 91s 129ms/step - loss: 0.5190 - binary_crossentropy: 0.5190 - val_loss: 0.5219 - val_binary_crossentropy: 0.5219
Epoch 4/5
704/704 [==============================] - 90s 128ms/step - loss: 0.5077 - binary_crossentropy: 0.5077 - val_loss: 0.5204 - val_binary_crossentropy: 0.5204
Epoch 5/5
704/704 [==============================] - 89s 127ms/step - loss: 0.4976 - binary_crossentropy: 0.4976 - val_loss: 0.5174 - val_binary_crossentropy: 0.5174



▶ Experiment 7: {'epochs': 5, 'learning_rate': 0.0005, 'dropout': 0.3, 'batch_size': 2048, 'embed_dim': 8, 'att_head_num': 4}
Epoch 1/5
352/352 [==============================] - 9s 25ms/step - loss: 0.6211 - binary_crossentropy: 0.6211 - val_loss: 0.5509 - val_binary_crossentropy: 0.5509
Epoch 2/5
352/352 [==============================] - 8s 23ms/step - loss: 0.5412 - binary_crossentropy: 0.5412 - val_loss: 0.5393 - val_binary_crossentropy: 0.5393
Epoch 3/5
352/352 [==============================] - 8s 24ms/step - loss: 0.5339 - binary_crossentropy: 0.5339 - val_loss: 0.5370 - val_binary_crossentropy: 0.5370
Epoch 4/5
352/352 [==============================] - 8s 23ms/step - loss: 0.5305 - binary_crossentropy: 0.5305 - val_loss: 0.5354 - val_binary_crossentropy: 0.5354
Epoch 5/5
352/352 [==============================] - 8s 24ms/step - loss: 0.5268 - binary_crossentropy: 0.5268 - val_loss: 0.5320 - val_binary_crossentropy: 0.5320



▶ Experiment 8: {'epochs': 5, 'learning_rate': 0.0005, 'dropout': 0.3, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 8}
Epoch 1/5
352/352 [==============================] - 36s 99ms/step - loss: 0.5845 - binary_crossentropy: 0.5845 - val_loss: 0.5420 - val_binary_crossentropy: 0.5420
Epoch 2/5
352/352 [==============================] - 35s 99ms/step - loss: 0.5377 - binary_crossentropy: 0.5377 - val_loss: 0.5392 - val_binary_crossentropy: 0.5392
Epoch 3/5
352/352 [==============================] - 35s 98ms/step - loss: 0.5315 - binary_crossentropy: 0.5315 - val_loss: 0.5320 - val_binary_crossentropy: 0.5320
Epoch 4/5
352/352 [==============================] - 35s 98ms/step - loss: 0.5248 - binary_crossentropy: 0.5248 - val_loss: 0.5294 - val_binary_crossentropy: 0.5294
Epoch 5/5
352/352 [==============================] - 36s 101ms/step - loss: 0.5202 - binary_crossentropy: 0.5202 - val_loss: 0.5273 - val_binary_crossentropy: 0.5273



▶ Experiment 9: {'epochs': 3, 'learning_rate': 0.0001, 'dropout': 0.7, 'batch_size': 1024, 'embed_dim': 16, 'att_head_num': 2}
Epoch 1/3
704/704 [==============================] - 11s 14ms/step - loss: 0.6457 - binary_crossentropy: 0.6457 - val_loss: 0.5892 - val_binary_crossentropy: 0.5892
Epoch 2/3
704/704 [==============================] - 10s 14ms/step - loss: 0.5575 - binary_crossentropy: 0.5575 - val_loss: 0.5442 - val_binary_crossentropy: 0.5442
Epoch 3/3
704/704 [==============================] - 13s 19ms/step - loss: 0.5374 - binary_crossentropy: 0.5374 - val_loss: 0.5399 - val_binary_crossentropy: 0.5399



▶ Experiment 10: {'epochs': 3, 'learning_rate': 0.0001, 'dropout': 0.7, 'batch_size': 1024, 'embed_dim': 16, 'att_head_num': 4}
Epoch 1/3
704/704 [==============================] - 18s 24ms/step - loss: 0.6289 - binary_crossentropy: 0.6289 - val_loss: 0.5591 - val_binary_crossentropy: 0.5591
Epoch 2/3
704/704 [==============================] - 16s 22ms/step - loss: 0.5440 - binary_crossentropy: 0.5440 - val_loss: 0.5412 - val_binary_crossentropy: 0.5412
Epoch 3/3
704/704 [==============================] - 16s 22ms/step - loss: 0.5356 - binary_crossentropy: 0.5356 - val_loss: 0.5390 - val_binary_crossentropy: 0.5390

 모든 실험 학습 완료.


## 평가

In [176]:
results = []

for i, (model, config) in enumerate(zip(trained_models, model_configs)):
    print(f"\n Evaluating model {i+1}: {config}")
    
    user_pred_info = {}
    mymodel_user_pred_info = test_model(model, test_df)

    for user, data_info in mymodel_user_pred_info.items():
        ranklist = sorted(data_info, key=lambda s: s[1], reverse=True)[:10]
        ranklist = list(dict.fromkeys([r[0] for r in ranklist]))
        user_pred_info[str(user)] = ranklist

    test_data = test_df[test_df['label'] == 1].groupby('user_id')['movie_id'].apply(list)

    mymodel_ndcg_result = {}
    mymodel_hitrate_result = {}

    for user, data_info in test_data.items():
        mymodel_pred = user_pred_info.get(str(user), [])[:10]
        testset = list(set(np.array(data_info).astype(int)))

        mymodel_ndcg_result[user] = get_NDCG(mymodel_pred, testset)
        mymodel_hitrate_result[user] = get_hit_rate(mymodel_pred, testset)

    ndcg = round(np.mean(list(mymodel_ndcg_result.values())), 5)
    hitrate = round(np.mean(list(mymodel_hitrate_result.values())), 5)

    print(f"결과: NDCG={ndcg}, Hitrate={hitrate}")

    results.append({
        "Config": f"Exp_{i+1}",
        "Epochs": config["epochs"],
        "LR": config["learning_rate"],
        "Dropout": config["dropout"],
        "Batch": config["batch_size"],
        "Embed": config["embed_dim"],
        "AttHead":config["att_head_num"],
        "NDCG": ndcg,
        "Hitrate": hitrate
    })

results_df = pd.DataFrame(results).sort_values(by="NDCG", ascending=False).reset_index(drop=True)


 Evaluating model 1: {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 2}
결과: NDCG=0.6616, Hitrate=0.63023

 Evaluating model 2: {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 4}
결과: NDCG=0.66144, Hitrate=0.63016

 Evaluating model 3: {'epochs': 5, 'learning_rate': 0.0001, 'dropout': 0.4, 'batch_size': 2048, 'embed_dim': 16, 'att_head_num': 8}
결과: NDCG=0.66154, Hitrate=0.6302

 Evaluating model 4: {'epochs': 3, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 32, 'att_head_num': 4}
결과: NDCG=0.66707, Hitrate=0.63379

 Evaluating model 5: {'epochs': 5, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 16, 'att_head_num': 8}
결과: NDCG=0.67171, Hitrate=0.63594

 Evaluating model 6: {'epochs': 5, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 32, 'att_head_num': 8}
결과: NDCG=0.67482, Hitrate=0.63843

 E

In [177]:
print("실험 결과 요약표")
display(results_df)

실험 결과 요약표


,Config,Epochs,LR,Dropout,Batch,Embed,AttHead,NDCG,Hitrate
0,Exp_6,5,0.0010,0.5,1024,32,8,0.67482,0.63843
1,Exp_5,5,0.0010,0.5,1024,16,8,0.67171,0.63594
2,Exp_4,3,0.0010,0.5,1024,32,4,0.66707,0.63379
3,Exp_8,5,0.0005,0.3,2048,16,8,0.66527,0.63177
4,Exp_9,3,0.0001,0.7,1024,16,2,0.66213,0.63042
5,Exp_7,5,0.0005,0.3,2048,8,4,0.66186,0.63037
6,Exp_1,5,0.0001,0.4,2048,16,2,0.66160,0.63023
7,Exp_3,5,0.0001,0.4,2048,16,8,0.66154,0.63020
8,Exp_10,3,0.0001,0.7,1024,16,4,0.66147,0.62991
9,Exp_2,5,0.0001,0.4,2048,16,4,0.66144,0.63016


## 베스트 모델 저장

In [181]:
# 1등 모델의 Config 값
best_exp_name = results_df.loc[0, "Config"]

# 1등 모델의 index
best_idx = int(best_exp_name.split("_")[1]) - 1

# 저장할 모델
best_model = trained_models[best_idx]
best_config = model_configs[best_idx]

# 저장 로직
np.save('./data/autoIntMLP_field_dims.npy', field_dims)
best_model.save_weights('./model/autoIntMLP_model_weights.h5')
joblib.dump(label_encoders, './data/autoIntMLP_label_encoders.pkl')

print(f"\nBest Model ({best_exp_name}) 저장 완료: {best_config}")


Best Model (Exp_6) 저장 완료: {'epochs': 5, 'learning_rate': 0.001, 'dropout': 0.5, 'batch_size': 1024, 'embed_dim': 32, 'att_head_num': 8}
